# Train and Evaluate a KGE model

In [3]:
# Install dicee 
!pip install dicee

In [1]:
from dicee import KGE, Execute
from dicee.config import Args

/upb/users/n/nkouagou/profiles/unix/cs/anaconda3/envs/daikiri/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /local/upb/users/n/nkouagou/profiles/unix/cs/anaconda3/envs/daikiri/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


# How to Train

In [2]:
# (1) Load Default Params
args=Args()
# (2) Select a Dataset and a KGE
args.path_dataset_folder="../KGs/UMLS"
args.model="FMult"#"AConEx"
args.embedding_dim=32
args.num_epochs=256
args.num_of_output_channels= 32
args.batch_size=1024
args.scoring_technique="NegSample"
args.neg_ratio = 1
args.eval_model="train_val_test"
args

In [3]:
!head ../KGs/UMLS/train.txt

acquired_abnormality	location_of	experimental_model_of_disease
anatomical_abnormality	manifestation_of	physiologic_function
alga	isa	entity
mental_or_behavioral_dysfunction	affects	experimental_model_of_disease
health_care_activity	associated_with	anatomical_abnormality
population_group	interacts_with	age_group
clinical_attribute	result_of	natural_phenomenon_or_process
body_part_organ_or_organ_component	location_of	biologic_function
biologically_active_substance	complicates	anatomical_abnormality
disease_or_syndrome	result_of	acquired_abnormality


In [4]:
executor=Execute(args)

In [5]:
report=executor.start()

Start time:2023-04-12 11:38:01.458296
*** Read or Load Knowledge Graph  ***
*** Reading ../KGs/UMLS/valid.txt with Pandas ***
Reading with pandas.read_csv with sep ** s+ ** ...
Took 0.0046 seconds | Current Memory Usage  522.3 in MB
*** Reading ../KGs/UMLS/test.txt with Pandas ***
Reading with pandas.read_csv with sep ** s+ ** ...
Took 0.0022 seconds | Current Memory Usage  522.4 in MB
*** Reading ../KGs/UMLS/train.txt with Pandas ***
Reading with pandas.read_csv with sep ** s+ ** ...
Took 0.0061 seconds | Current Memory Usage  522.91 in MB

Concatenating data to obtain index...
Done !

Creating a mapping from entities to integer indexes...
Done !

Done ! 0.006 seconds

Done !

Done !

Took 0.0136 seconds | Current Memory Usage  523.92 in MB
Data Type conversion...
Submit er-vocab, re-vocab, and ee-vocab via  ProcessPoolExecutor...
Preprocessing took: 0.048 seconds

------------------- Description of Dataset ../KGs/UMLS -------------------
Number of entities:135
Number of relations:46


ValueError: 

In [7]:
report

{'num_train_triples': 10432,
 'num_entities': 135,
 'num_relations': 92,
 'EstimatedSizeMB': 0.13260650634765625,
 'NumParam': 138784,
 'path_experiment_folder': '/home/demir/Desktop/Softwares/dice-embeddings/examples/Experiments/2023-04-05 12:59:36.749715',
 'Runtime': 28.33254861831665,
 'Train': {'H@1': 0.8515145705521472,
  'H@3': 0.9526457055214724,
  'H@10': 0.9818826687116564,
  'MRR': 0.9049869811033961},
 'Val': {'H@1': 0.6694785276073619,
  'H@3': 0.8351226993865031,
  'H@10': 0.9386503067484663,
  'MRR': 0.7677471996019155},
 'Test': {'H@1': 0.6717095310136157,
  'H@3': 0.8562783661119516,
  'H@10': 0.9402420574886535,
  'MRR': 0.771550307601881}}

# How to Eval

In [8]:
pre_trained_model=KGE(path=report['path_experiment_folder'])

Loading model model.pt... Done! It took 0.002
Loading entity and relation indexes... Done! It took 0.0002
Loading indexed training data...


In [9]:
pre_trained_model.topk(head_entity='behavior',relation='associated_with',k=10)

(tensor([0.9401, 0.9305, 0.8677, 0.8673, 0.8347, 0.6596, 0.4561, 0.3340, 0.1839,
         0.1810]),
 ['family_group',
  'professional_or_occupational_group',
  'population_group',
  'group',
  'age_group',
  'patient_or_disabled_group',
  'carbohydrate_sequence',
  'geographic_area',
  'sign_or_symptom',
  'qualitative_concept'])

In [10]:
pre_trained_model.find_missing_triples(confidence=0.8,
                                       entities=['behavior'],
                                       relations=['associated_with'],topk=10)

Number of entities:135 	 Number of relations:92
Finding missing triples..
Number of found missing triples: 1


{('behavior', 'associated_with', 'age_group')}

In [11]:
# Data is already saved
!ls Experiments

'2023-04-05 12:59:36.749715'


# How to deploy

In [ ]:
# Pretrained model can be used with anyone
pre_trained_model.deploy(share=True,top_k=10)